# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

First, let's connect to our SQLite database containing music store data.


In [ ]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Now we'll create a tool that allows the agent to execute SQL queries against the database.


In [ ]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

We'll define a runtime context schema to track whether the current user is an employee.


In [ ]:
import { z } from "zod";

const RuntimeContext = z.object({
    isEmployee: z.boolean()
});
type RuntimeContext = z.infer<typeof RuntimeContext>;

Here's our base system prompt with a placeholder (`%s`) that will be dynamically filled based on the user's role.


In [ ]:
const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
%s
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.`

This middleware dynamically adjusts the system prompt: non-employees get restricted to music-related tables only.


In [ ]:
import { format } from "node:util";
import { dynamicSystemPromptMiddleware, type ModelRequest, type AgentState, type Runtime } from "langchain";

const dynamicSystemPrompt = dynamicSystemPromptMiddleware((state: AgentState, runtime: Runtime<RuntimeContext>) => {
    return !runtime.context.isEmployee
        ? format(SYSTEM, "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track.")
        : format(SYSTEM, "")
})

Let's create the agent with our SQL tool, system prompt, context schema, and dynamic middleware.


In [ ]:
import * as setup from "./setup.ts";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5-mini",
    tools: [executeSQL],
    contextSchema: RuntimeContext,
    middleware: [dynamicSystemPrompt]
})

## Testing with Different Roles

Let's test as a **non-employee** first. The agent should refuse access to customer and invoice tables.


In [ ]:
const stream = await agent.stream({
    messages: "What is the most costly purchase by Frank Harris?",
}, {
    streamMode: "values",
    context: { isEmployee: false }
})

for await (const step of stream) {
    displayMessage(step.messages.at(-1))
}

Now let's test as an **employee**. This time, the agent has full access and can query all tables.


In [ ]:
const stream = await agent.stream({
    messages: "What is the most costly purchase by Frank Harris?",
}, {
    streamMode: "values",
    context: { isEmployee: true }
})

for await (const step of stream) {
    displayMessage(step.messages.at(-1))
}